In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import the data using pandas
housing_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
# show first five rows to make sure data loads correctly
housing_df.head()

In [ ]:
# print the rows and columns of the dataset
print(housing_df.shape)
# show me what percentage of the data is missing for each column in dataset
(housing_df.isnull().sum()/housing_df.isnull().count()).sort_values(ascending=False)

In [ ]:
# perform descriptive statistics
describe = housing_df["SalePrice"].describe()
# draw some conclusions
print("The price range in which 68% of the prices lie is from: ${} - ${}".format(round((describe[1]-describe[2]),2),round((describe[1]+describe[2]),2)))
describe

In [ ]:
# show a distribution and histogram plot of the sales price using seaborn
ax = sns.displot(housing_df, x="SalePrice", kde=True)
ax.set(title="Sales Price Distribution")

In [ ]:
# read in data and display
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train_df.head()

In [ ]:
# show what % of data is missing in descending order
# since Id shows no missing data any column beneath it has all its' data
print(train_df.shape)
(train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False).iloc[0:20]

In [ ]:
# obtain the column names with string data types
categorical_data = []

for i in range(len(train_df.dtypes)):
    if "64" not in str(train_df.dtypes[i]):
        categorical_data.append(train_df.columns[i])

# Reading the data description doc I see some of the numerical data represents categorical information
# This fact is accounted for here and is added to the categorical dataset
cat_data = ["MSSubClass","OverallQual","OverallCond","YearBuilt","YearRemodAdd",
           "BsmtFullBath","BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr",
           "KitchenAbvGr","TotRmsAbvGrd","GarageYrBlt","MoSold","YrSold"]

categorical_data += cat_data

# perform descriptive statistics on categorical data
train_df[categorical_data].astype('category').describe().T

In [ ]:
# obtain the column names with numerical data types
numerical_data = []

for i in range(len(train_df.dtypes)):
    if "64" in str(train_df.dtypes[i]):
        numerical_data.append(train_df.columns[i])
        
# remove Id column as it provides no insight
numerical_data.remove("Id")

# remove the categorical data we found previously
for num_data in numerical_data:
        if num_data in categorical_data:
            numerical_data.remove(num_data)
numerical_data.remove("KitchenAbvGr")
numerical_data.remove("OverallCond")
numerical_data.remove("BsmtHalfBath")
numerical_data.remove("HalfBath")

# perform descriptive statistics on numerical data
train_df[numerical_data].describe().T

In [ ]:
# As we can see there are little to no Gravel street types for Classifications RM, FV, and RH
sns.catplot(x="MSZoning", y="SalePrice", hue="Street",kind="violin",split=True, data=train_df)
plt.title("Distribution of Sale Price over Street Type per Zoning Classification")

In [ ]:
# Using Alley Type instead of Street Type we obtain nearly the opposite shading
# There is virtually no Gravel types for Zones C, FV, and RH
sns.catplot(x="MSZoning", y="SalePrice", hue="Alley",kind="violin",split=True, data=train_df)
plt.title("Distribution of Sale Price over Alley Type per Zoning Classification")

In [ ]:
# group sales by year and month then count sales
date_df = train_df.groupby("YrSold")["MoSold"].value_counts()

# plot results
date_df.sort_index().plot(kind="bar",figsize=(15,8),title="Sales Over Time",
                          xlabel="Year,Month",ylabel="Houses Sold")
# draw insights
print("As we can see there is a cyclical nature to the sale of homes")

In [ ]:
# group sales by Year and Month then count sales
date_sold = train_df.groupby("YrSold")["MoSold"].value_counts().unstack().fillna(0)

# convert month number to month name for readability    
import datetime
mon_name = []
for mon_num in date_sold.T.index:
    name = datetime.datetime.strptime(str(mon_num),"%m").strftime("%b")
    mon_name.append(name)

In [ ]:
# sum number of houses sold in each year for each month
yoy = round(date_sold.sum())

explode = [0,0,0,0,0.06,0.06,0.06,0,0,0,0,0]
yoy.groupby(['MoSold']).sum().plot(kind='pie',figsize=(15,10),autopct='%1.1f%%',labels=mon_name,explode=explode)
plt.title("% of Sales Per Month")
plt.ylabel("")

sales_percent = 100*round((date_sold.sum().iloc[4:7].sum()/date_sold.sum().sum()),3)
print("From this area graph we can see {}% of total sales occur between the 3 months of May, June, and July.".format(sales_percent))

In [ ]:
# create a dataframe of dates and selling prices
revenue_df = train_df[["YrSold","MoSold","SalePrice"]]
# group dates by year and month then sum over all sales revenue
revenue_groups = revenue_df.groupby(["YrSold","MoSold"]).sum()
# plot results
ax = revenue_groups.plot(
kind="bar",figsize=(15,8),title="Revenue Over Time",
xlabel="Year,Month",ylabel="Revenue ($10M)",yticks=range(0,13000000,10**6))
print("As we can expect, revenue is related to sales volume.")

In [ ]:
# sum all sales revenue for each month for all years
monthly_rev = revenue_groups[["SalePrice"]].groupby(["MoSold"]).sum()
# sales revenue as a % from May, June, July
rev_percent = 13.3+17.0+16.5
# draw insights and plot results
print("As expected, we can see that {}% of total revenue comes from May, June, and July".format(rev_percent))
monthly_rev.plot(kind='pie',figsize=(15,10),legend=False,autopct='%1.1f%%',labels=mon_name,explode=explode,subplots=True)
plt.title("% of Total Revenue per Month")
plt.ylabel("")

In [ ]:
# get amount of houses sold per date
date_df_sort = date_df.sort_index()
# add amount of houses sold to revenue dataframe
revenue_groups["AmountSold"] = date_df_sort.values
# calculate average selling price of home per date and round to nearest cent
revenue_groups["AvgPrice"] = round((revenue_groups["SalePrice"]/revenue_groups["AmountSold"]),2)
# plot results
avgSales_df = revenue_groups[["AvgPrice"]]
ax = avgSales_df.plot(
kind="bar",figsize=(15,8),title="Avg Revenue per House Over Time",
xlabel="Year,Month",ylabel="Revenue ($)",ylim=(120000,230000))
# draw insights
print("Perhaps counter-intuitively, while sales volume and revenue typically peak in the "+
     "month of June, the average price of a home peaks in September.")

In [ ]:
# How does a pool affect home price?

# create dataframe containing pool data
pool_df = train_df[["PoolQC","PoolArea","SalePrice"]].replace(np.nan,"None")
# create dataframe containing houses with pools
yes_pool = pool_df[pool_df["PoolQC"] != "None"]
# create dataframe containing houses without pools
no_pool = pool_df[train_df["PoolQC"] == "None"]
# plot pool area vs price per pool quality
sns.scatterplot(data=yes_pool, x="PoolArea", y="SalePrice", hue="PoolQC")
plt.title("Sale Price over Pool Area per PoolQC")
# draw insights
print("It seems there are too few houses with pools to draw any concrete conclusions, " +
     "however, we may be able to say houses with larger and higher quality pools have " +
     "higher retail value.")

In [ ]:
# obtain df with lot information
area_df = train_df[["LotArea","LotShape","LotFrontage","SalePrice","LandContour",
                    "LotConfig","LandSlope"]]
# categorical columns
lot_categories = ["LandContour","LotConfig","LandSlope","LotShape"]

# create a grid plot for each categorical value
for cat in lot_categories:
    quant_area = area_df[["LotArea","LotFrontage","SalePrice",cat]]
    g = sns.pairplot(quant_area, hue=cat, height=2.5,kind="scatter",diag_kind="kde")
    g.fig.suptitle("Category: " + cat, y=1.08)

In [ ]:
# create a df containing neighborhood categories
neighborhood = train_df[["Neighborhood"]]
# count how many times each neighborhood appears, rank, then plot
neighborhood.value_counts("Neighborhood",ascending=True).plot(kind="barh",figsize=(10,8))
plt.title("Frequency of Each Neighborhood")
plt.xlabel("Counts")

In [ ]:
# dataframe to gain insights of home value of different neighborhoods
n_worth_df = train_df[["SalePrice","Neighborhood","YearRemodAdd","MasVnrType"]]
# plot year modeled against sale price for each neighborhood accounting for Masonry Veneer
g = sns.FacetGrid(n_worth_df, col="Neighborhood",col_wrap=5,hue="MasVnrType")
g.map(sns.scatterplot, "YearRemodAdd", "SalePrice", alpha=.7)
g.add_legend()
# insights
print("The more recent the remodel of a house the more value the home has.")

In [ ]:
# create df of MSZoning and MSSubClass
MS_df = train_df[["MSZoning","MSSubClass"]]
# create data frame of counts by SubClass of Zoning
ms_df = MS_df.groupby("MSSubClass")["MSZoning"].value_counts().unstack().fillna(0)
# sort by RL of Zoning and create a stacked barh graph
ms_df.sort_values("RL").plot.barh(stacked=True,figsize=(10,5))
# plotting this way results in a more digestible graph
plt.title("Zoning Makeup of Subclass")
plt.xlabel("Counts")
# draw insights
print("We can see Residential Low Density makes up the bulk of most Dwelling Types.")

In [ ]:
revenue_df = train_df[["YrSold","MoSold","SalePrice"]]
# group dates by year and month
revenue_df.groupby("YrSold")["MoSold"].count()
#revenue_df['date'] = revenue_df['MoSold'].map(str)+ '-' +revenue_df['YrSold'].map(str)
#revenue_df['date'] = pd.to_datetime(revenue_df['date'], format='%m-%Y').dt.strftime('%m-%Y')
# plot
#fig, ax = plt.subplots()
#plt.plot_date(revenue_df['date'], revenue_df['SalePrice'])
#plt.show()